**CODIGO**

*   Elemento de lista
*   Elemento de lista



In [3]:
!pip install efficient_apriori
!pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5974 sha256=0c759f3e1307e45d0bf7c0f15e430d374fda06c6ed3c648b43f90f2a73e39808
  Stored in directory: /root/.cache/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori


In [4]:
import pandas as pd
import numpy as np
from apyori import apriori

In [5]:
from google.colab import files 
uploaded = files.upload()

Saving store_data.csv to store_data.csv


In [41]:
df=pd.read_csv('store_data.csv')
df.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df.isnull().sum()

shrimp               0
almonds              0
avocado              0
vegetables mix       0
green grapes         0
whole weat flour     0
yams                 0
cottage cheese       0
energy drink         0
tomato juice         0
low fat yogurt       0
green tea            0
honey                0
salad                0
mineral water        0
salmon               0
antioxydant juice    0
frozen smoothie      0
spinach              0
olive oil            0
dtype: int64

In [43]:
df.fillna(0,inplace=True)

In [44]:
df

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
2,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
3,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,low fat yogurt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,butter,light mayo,fresh bread,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7496,burgers,frozen vegetables,eggs,french fries,magazines,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7497,chicken,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
7498,escalope,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [45]:
#convertimos la data en lista 
transactions = []
for i in range(0,len(df)):
    transactions.append([str(df.values[i,j]) for j in range(0,20) if str(df.values[i,j])!='0'])

In [46]:
transactions[0]

['burgers', 'meatballs', 'eggs', '0.0']

In [47]:
rules = apriori(transactions,min_support=0.003,min_confidance=0.2,min_lift=3,min_length=2)

In [48]:
rules

<generator object apriori at 0x7f7de8b78bd0>

In [49]:
# all rules need to be converted in a list..
Results = list(rules)
Results


[RelationRecord(items=frozenset({'brownies', 'cottage cheese'}), support=0.0034666666666666665, ordered_statistics=[OrderedStatistic(items_base=frozenset({'brownies'}), items_add=frozenset({'cottage cheese'}), confidence=0.10276679841897234, lift=3.2384495300096323), OrderedStatistic(items_base=frozenset({'cottage cheese'}), items_add=frozenset({'brownies'}), confidence=0.10924369747899158, lift=3.2384495300096323)]),
 RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004533333333333334, ordered_statistics=[OrderedStatistic(items_base=frozenset({'chicken'}), items_add=frozenset({'light cream'}), confidence=0.07555555555555557, lift=4.8433048433048445), OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.2905982905982906, lift=4.843304843304844)]),
 RelationRecord(items=frozenset({'escalope', 'mushroom cream sauce'}), support=0.005733333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset({'escalope'}

In [86]:
#convert result in a dataframe for further operation...
df_results = pd.DataFrame(Results)

In [87]:
df_results.head()

,items,support,ordered_statistics
0,"(brownies, cottage cheese)",0.003467,"[((brownies), (cottage cheese), 0.102766798418..."
1,"(chicken, light cream)",0.004533,"[((chicken), (light cream), 0.0755555555555555..."
2,"(escalope, mushroom cream sauce)",0.005733,"[((escalope), (mushroom cream sauce), 0.072268..."
3,"(escalope, pasta)",0.005867,"[((escalope), (pasta), 0.07394957983193277, 4...."
4,"(fresh bread, tomato juice)",0.004267,"[((fresh bread), (tomato juice), 0.09907120743..."


In [88]:
#Separamos el support, lo usaremos luego..... 

support = df_results.support

In [89]:
#Separamos los datos de la columna  ordered_statistics 

first_values = []
second_values = []
third_values = []
fourth_value = []

for i in range(df_results.shape[0]):
    single_list = df_results['ordered_statistics'][i][0]
    first_values.append(list(single_list[0]))
    second_values.append(list(single_list[1]))
    third_values.append(single_list[2])
    fourth_value.append(single_list[3])

In [90]:
#ejecutamos todos los valores en un dataframe

lhs = pd.DataFrame(first_values)
rhs= pd.DataFrame(second_values)
confidance=pd.DataFrame(third_values,columns=['Confidance'])
lift=pd.DataFrame(fourth_value,columns=['lift'])



In [91]:
#Unimos todo: 

df_final = pd.concat([lhs,rhs,support,confidance,lift], axis=1)
df_final

,0,1,0,1,2,3,support,Confidance,lift
0,brownies,None,cottage cheese,None,None,None,0.003467,0.102767,3.238450
1,chicken,None,light cream,None,None,None,0.004533,0.075556,4.843305
2,escalope,None,mushroom cream sauce,None,None,None,0.005733,0.072269,3.790327
3,escalope,None,pasta,None,None,None,0.005867,0.073950,4.700185
4,fresh bread,None,tomato juice,None,None,None,0.004267,0.099071,3.273278
...,...,...,...,...,...,...,...,...,...
183,ground beef,pancakes,0.0,spaghetti,mineral water,None,0.003067,0.211009,3.532520
184,ground beef,None,0.0,spaghetti,mineral water,tomatoes,0.003067,0.031208,3.343671
185,olive oil,None,milk,0.0,spaghetti,mineral water,0.003333,0.050710,3.223089
186,mineral water,milk,0.0,shrimp,spaghetti,None,0.003067,0.063889,3.013627


In [92]:
'''
 we have some of place only 1 item in lhs and some place 3 or more so we need to a proper represenation for User to understand. 
 removing none with ' ' extra so when we combine three column in 1 then only 1 item will be there with spaces which is proper rather than none.
 example : coffee,none,none which converted to coffee, ,
'''
df_final.fillna(value=' ', inplace=True) 





In [93]:
df_final.columns = ['lhs',1,2,3,0,'rhs','support','confidance','lift']

In [94]:
df_final['lhs'] = df_final['lhs']+str(", ")+df_final[1]+str(", ")+df_final[2]

In [95]:
df_final

,lhs,1,2,3,0,rhs,support,confidance,lift
0,"brownies, , cottage cheese",,cottage cheese,,,,0.003467,0.102767,3.238450
1,"chicken, , light cream",,light cream,,,,0.004533,0.075556,4.843305
2,"escalope, , mushroom cream sauce",,mushroom cream sauce,,,,0.005733,0.072269,3.790327
3,"escalope, , pasta",,pasta,,,,0.005867,0.073950,4.700185
4,"fresh bread, , tomato juice",,tomato juice,,,,0.004267,0.099071,3.273278
...,...,...,...,...,...,...,...,...,...
183,"ground beef, pancakes, 0.0",pancakes,0.0,spaghetti,mineral water,,0.003067,0.211009,3.532520
184,"ground beef, , 0.0",,0.0,spaghetti,mineral water,tomatoes,0.003067,0.031208,3.343671
185,"olive oil, , milk",,milk,0.0,spaghetti,mineral water,0.003333,0.050710,3.223089
186,"mineral water, milk, 0.0",milk,0.0,shrimp,spaghetti,,0.003067,0.063889,3.013627


In [96]:
df_final.drop(columns=[1,2,3,0],inplace=True)

In [98]:
df_final

,lhs,rhs,support,confidance,lift
0,"brownies, , cottage cheese",,0.003467,0.102767,3.238450
1,"chicken, , light cream",,0.004533,0.075556,4.843305
2,"escalope, , mushroom cream sauce",,0.005733,0.072269,3.790327
3,"escalope, , pasta",,0.005867,0.073950,4.700185
4,"fresh bread, , tomato juice",,0.004267,0.099071,3.273278
...,...,...,...,...,...
183,"ground beef, pancakes, 0.0",,0.003067,0.211009,3.532520
184,"ground beef, , 0.0",tomatoes,0.003067,0.031208,3.343671
185,"olive oil, , milk",mineral water,0.003333,0.050710,3.223089
186,"mineral water, milk, 0.0",,0.003067,0.063889,3.013627


# Seccion de asignacion

In [ ]:
df2=pd.get_dummies(df)

In [ ]:
df2

,olive oil,shrimp_almonds,shrimp_antioxydant juice,shrimp_asparagus,shrimp_avocado,shrimp_babies food,shrimp_bacon,shrimp_barbecue sauce,shrimp_black tea,shrimp_blueberries,shrimp_body spray,shrimp_bramble,shrimp_brownies,shrimp_bug spray,shrimp_burger sauce,shrimp_burgers,shrimp_butter,shrimp_cake,shrimp_candy bars,shrimp_carrots,shrimp_cauliflower,shrimp_cereals,shrimp_champagne,shrimp_chicken,shrimp_chili,shrimp_chocolate,shrimp_chocolate bread,shrimp_chutney,shrimp_cider,shrimp_clothes accessories,shrimp_cookies,shrimp_cooking oil,shrimp_corn,shrimp_cottage cheese,shrimp_cream,shrimp_dessert wine,shrimp_eggplant,shrimp_eggs,shrimp_energy bar,shrimp_energy drink,...,salad_strawberries,salad_tomato juice,salad_toothpaste,salad_yogurt cake,mineral water_0,mineral water_candy bars,mineral water_cereals,mineral water_clothes accessories,mineral water_cookies,mineral water_fresh bread,mineral water_green tea,mineral water_hand protein bar,mineral water_hot dogs,mineral water_low fat yogurt,mineral water_magazines,mineral water_mint,mineral water_mushroom cream sauce,mineral water_pancakes,mineral water_salmon,mineral water_tomato juice,mineral water_toothpaste,mineral water_yogurt cake,mineral water_zucchini,salmon_0,salmon_antioxydant juice,salmon_cake,salmon_chocolate,salmon_frozen smoothie,salmon_magazines,salmon_protein bar,salmon_sparkling water,antioxydant juice_0,antioxydant juice_french fries,antioxydant juice_frozen smoothie,frozen smoothie_0,frozen smoothie_protein bar,frozen smoothie_spinach,spinach_0,spinach_cereals,spinach_mayonnaise
0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
2,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
3,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
4,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
7496,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
7497,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0
7498,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0


In [ ]:
min_support = 7/len(df) 

In [ ]:
min_support

0.0009333333333333333

In [ ]:
min_confidence = 0

In [ ]:
itemsets, rules = apriori(df, min_support=min_support, min_confidence=min_confidence)


ValueError: ignored

range(1, 10)

In [23]:
from efficient_apriori import apriori

import pandas as pd
